 # using STResNet

In [6]:
## headings
"""
made by weiyw @ 2019-03-08
made to use the deep Spatio-temporal Residual Network
"""
# from __future__ import print_function
# from preprocessing import *
# import numpy as np
# np.random.seed(2333)  # for reproducibility


import os
import time
import struct
import numpy as np
import h5py

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from STResNet import stresnet
import metrics as metrics

from PIL import Image
import matplotlib.pyplot as plt

In [15]:
def get_min_nonzero_value(data_in, init_min = 1):
    min_value = init_min
    for bb in data_in:
        for aa in bb:
            if abs(aa) < min_value and aa != 0:
                min_value = abs(aa)
    return min_value

In [ ]:
def prepare_data(filepath, all_times, data_step, data_stride):
    X = []
    y = []
    for i in range(all_times):
        count_X = range(i*data_stride, i*data_stride + data_step)
        for j in count_X:
            img = Image.open(filepath + "/" + str(j+1)+".png")
            data = np.resize(np.asarray(img), (330, 580, 4))#np.asarray(img)
            X.append(data[:,:,0])
        count_y = i*data_stride + data_step
        img = Image.open(filepath + "/" + str(count_y+1)+".png")
        data = np.resize(np.asarray(img), (330, 580, 4))#np.asarray(img)
        y.append(data[:,:,0])
    X = np.array(X, dtype='float32')
    y = np.array(y, dtype='float32')
    return X, y

In [4]:
def get_bindata(data_path, data_index, data_phase, nt, nr):
    # data_index start from 1
    model_num = int( ( int(data_index) - 1 ) / 51) + 1
    source_num = ( int(data_index) - 1 ) % 51 + 1
    data_name = data_path + "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
    print(data_name)
    
    out_data = np.empty((nt,nr))
    FA = open(data_name, "rb")
    FA.seek(3232,0)
    for tt in range(nt):
        for rr in range(nr):
            data = FA.read(4)
            data_float = struct.unpack("f", data)[0]
            out_data[tt][rr] = data_float
     #np.shape(out_data)   # (4000,301)
#     cut_data = out_data[0:4000, 8:401]
    return out_data
    

In [7]:
## load in data - path
# data_path = "./mydata/first_test/"
data_path = "/media/wywdisk/VSPdata/data/layer2_1-90_rightPML/vz"

datapath_list = os.listdir(data_path)
datapath_list.sort()
totalNum = len(datapath_list)
print(totalNum)

# indexNum = 1
# data_name = data_path + str(indexNum) + ".png"
# print(data_name)
data_index = 1
data_phase = "vz"
nt = 4000
nr = 300
this_data = get_bindata(data_path, data_index, data_phase, nt, nr)

4590
/media/wywdisk/VSPdata/data/layer2_1-90_rightPML/vz/model1source1vz.bin


In [18]:
target = "this_data"
print "The shape of ", target, "\tis", eval(target).shape
print "The max:\t",  eval(target).max(), "\tThe min:\t",  eval(target).min()
print "The mean:\t",  eval(target).mean(), "\tThe std:\t",  eval(target).std()
print "The absolute minimal value is\t", get_min_nonzero_value(this_data)

The shape of  this_data 	is (4000, 300)
The max:	6.42170998617e-05 	The min:	-6.66249470669e-05
The mean:	6.86736828383e-15 	The std:	6.77819107633e-07
The absolute mininal value is	1.40129846432e-45


In [ ]:
img = Image.open(data_name)

In [ ]:
img.data

In [ ]:
# 1）数据组合成网络需要的形式
# 2）
